In [ ]:
# !pip install datasets
# !pip install transformers

In [ ]:
# !conda install pytorch::pytorch torchvision torchaudio -c pytorch

In [1]:
# Pretty print
from pprint import pprint
# Datasets load_dataset function
from datasets import load_dataset
# Transformers Autokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
# Standard PyTorch DataLoader
from torch.utils.data import DataLoader

In [ ]:
import nltk
import os
import re
import math
import operator
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

In [3]:
hupd_dict = load_dataset('HUPD/hupd',
    name='sample',
    data_files="https://huggingface.co/datasets/HUPD/hupd/blob/main/hupd_metadata_2022-02-22.feather",
    icpr_label=None,
    train_filing_start_date='2016-01-01',
    train_filing_end_date='2016-01-21',
    val_filing_start_date='2016-01-22',
    val_filing_end_date='2016-01-31',
)

print('Loading is done!')

Loading is done!


In [4]:
all_claims = []
for i in range(hupd_dict['train'].shape[0]):
    claims_text = hupd_dict['train'][i]['claims']
    all_claims.append(claims_text)

In [5]:
Stopwords = set(stopwords.words('english'))
wordlemmatizer = WordNetLemmatizer()


please change the index here:
- example_index and org_claim


In [37]:
# generating summaries
# example_index = 5  # Change this to any index you want to summarize
# example_text = all_claims[example_index]
dirty_claim = hupd_dict['train'][1]['claims'] # Change this to any index you want to summarize same with "example_index"
# print("Original claim:\n", dirty_claim)

## TFIDF readymade extractive summarization

In [ ]:
def lemmatize_words(words):
    lemmatized_words = []
    for word in words:
       lemmatized_words.append(wordlemmatizer.lemmatize(word))
    return lemmatized_words
def stem_words(words):
    stemmed_words = []
    for word in words:
       stemmed_words.append(stemmer.stem(word))
    return stemmed_words
def remove_special_characters(text):
    regex = r'[^a-zA-Z0-9\s]'
    text = re.sub(regex,'',text)
    return text
def freq(words):
    words = [word.lower() for word in words]
    dict_freq = {}
    words_unique = []
    for word in words:
       if word not in words_unique:
           words_unique.append(word)
    for word in words_unique:
       dict_freq[word] = words.count(word)
    return dict_freq
def pos_tagging(text):
    pos_tag = nltk.pos_tag(text.split())
    pos_tagged_noun_verb = []
    for word,tag in pos_tag:
        if tag == "NN" or tag == "NNP" or tag == "NNS" or tag == "VB" or tag == "VBD" or tag == "VBG" or tag == "VBN" or tag == "VBP" or tag == "VBZ":
             pos_tagged_noun_verb.append(word)
    return pos_tagged_noun_verb
def tf_score(word,sentence):
    freq_sum = 0
    word_frequency_in_sentence = 0
    len_sentence = len(sentence)
    for word_in_sentence in sentence.split():
        if word == word_in_sentence:
            word_frequency_in_sentence = word_frequency_in_sentence + 1
    tf =  word_frequency_in_sentence/ len_sentence
    return tf
def idf_score(no_of_sentences,word,sentences):
    no_of_sentence_containing_word = 0
    for sentence in sentences:
        sentence = remove_special_characters(str(sentence))
        sentence = re.sub(r'\d+', '', sentence)
        sentence = sentence.split()
        sentence = [word for word in sentence if word.lower() not in Stopwords and len(word)>1]
        sentence = [word.lower() for word in sentence]
        sentence = [wordlemmatizer.lemmatize(word) for word in sentence]
        if word in sentence:
            no_of_sentence_containing_word = no_of_sentence_containing_word + 1
    idf = math.log10(no_of_sentences/no_of_sentence_containing_word)
    return idf
def tf_idf_score(tf,idf):
    return tf*idf
def word_tfidf(dict_freq,word,sentences,sentence):
    word_tfidf = []
    tf = tf_score(word,sentence)
    idf = idf_score(len(sentences),word,sentences)
    tf_idf = tf_idf_score(tf,idf)
    return tf_idf
def sentence_importance(sentence,dict_freq,sentences):
     sentence_score = 0
     sentence = remove_special_characters(str(sentence)) 
     sentence = re.sub(r'\d+', '', sentence)
     pos_tagged_sentence = [] 
     no_of_sentences = len(sentences)
     pos_tagged_sentence = pos_tagging(sentence)
     for word in pos_tagged_sentence:
          if word.lower() not in Stopwords and word not in Stopwords and len(word)>1: 
                word = word.lower()
                word = wordlemmatizer.lemmatize(word)
                sentence_score = sentence_score + word_tfidf(dict_freq,word,sentences,sentence)
     return sentence_score

In [ ]:
tokenized_sentence = sent_tokenize(dirty_claim)
text = remove_special_characters(str(dirty_claim))
text = re.sub(r'\d+', '', dirty_claim)
tokenized_words_with_stopwords = word_tokenize(dirty_claim)
tokenized_words = [word for word in tokenized_words_with_stopwords if word not in Stopwords]
tokenized_words = [word for word in tokenized_words if len(word) > 1]
tokenized_words = [word.lower() for word in tokenized_words]
tokenized_words = lemmatize_words(tokenized_words)
word_freq = freq(tokenized_words)
input_user = int(input('Percentage of information to retain(in percent):'))
no_of_sentences = int((input_user * len(tokenized_sentence))/100)
print(no_of_sentences)
c = 1
sentence_with_importance = {}
for sent in tokenized_sentence:
    sentenceimp = sentence_importance(sent,word_freq,tokenized_sentence)
    sentence_with_importance[c] = sentenceimp
    c = c+1
sentence_with_importance = sorted(sentence_with_importance.items(), key=operator.itemgetter(1),reverse=True)
cnt = 0
summary = []
sentence_no = []
for word_prob in sentence_with_importance:
    if cnt < no_of_sentences:
        sentence_no.append(word_prob[0])
        cnt = cnt+1
    else:
      break
sentence_no.sort()
cnt = 1
for sentence in tokenized_sentence:
    if cnt in sentence_no:
       summary.append(sentence)
    cnt = cnt+1
summary = " ".join(summary)
print("\n")
print("Summary:")
print(summary)
outF = open('summary.txt',"w")
outF.write(summary)

---

## TF-IDF Maths only calculation

reference from : https://towardsdatascience.com/text-summarization-using-tf-idf-e64a0644ace3

Background

TF-IDF is made up of two algorithms:
- Term Frequency : how common a word is
    - calculation: </br>
    <code> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document) </code>
- Inverse Document Frequency : how unique a word is
    - calculation: </br>
        <code> IDF(t) = log_e(Total number of documents / Number of documents with term t in it) </code>


### Sentence Tokenization

In [38]:
dirty_sentence = sent_tokenize(dirty_claim)
total_docs = len(dirty_sentence)

In [39]:
for i, sentence in enumerate(dirty_sentence):
    print(f"Sentence {i+1}/{total_docs}: {sentence}")

Sentence 1/41: 1.
Sentence 2/41: A method comprising: using a first reader to take a first reading of an inherent disorder feature of a tag; using at least a second reader to take at least a second reading of the inherent disorder feature of the tag; matching the first reading with at least the second reading; determining one or more acceptance criteria, wherein at least one of the acceptance criteria is based on whether the first reading and the second reading match within a predetermined threshold; accepting the tag if the acceptance criteria are met; and recording a fingerprint for the tag if the tag was accepted.
Sentence 3/41: 2.
Sentence 4/41: The method of claim 1, wherein determining one or more acceptance criteria further comprises: determining an acceptance criterion based on an individual reading.
Sentence 5/41: 3.
Sentence 6/41: The method of claim 2, wherein determining an acceptance criterion based on an individual reading comprises determining an acceptance criterion bas

because the claim sentences are divided into two parts: one is number and one is claims, <code> combined_sentence </code> array is created to combine those divided part into one sentence

In [40]:
combined_sentences = []
i = 0

while i < len(dirty_sentence):
    current_sentence = dirty_sentence[i]
    next_index = i + 1

    if current_sentence.replace('.', '').isdigit():
        if next_index < len(dirty_sentence) and not dirty_sentence[next_index].replace('.', '').isdigit():
            current_sentence += " " + dirty_sentence[next_index]
            i = next_index 
    combined_sentences.append(current_sentence)
    i += 1

for index, sentence in enumerate(combined_sentences):
    print(f"Sentence {index+1}/{len(combined_sentences)}: {sentence}")

Sentence 1/22: 1. A method comprising: using a first reader to take a first reading of an inherent disorder feature of a tag; using at least a second reader to take at least a second reading of the inherent disorder feature of the tag; matching the first reading with at least the second reading; determining one or more acceptance criteria, wherein at least one of the acceptance criteria is based on whether the first reading and the second reading match within a predetermined threshold; accepting the tag if the acceptance criteria are met; and recording a fingerprint for the tag if the tag was accepted.
Sentence 2/22: 2. The method of claim 1, wherein determining one or more acceptance criteria further comprises: determining an acceptance criterion based on an individual reading.
Sentence 3/22: 3. The method of claim 2, wherein determining an acceptance criterion based on an individual reading comprises determining an acceptance criterion based on a strength of a signal in at least one 

### TF Calculation

<strong>Frequency Matrix Calcuation </strong> </br>
- An empty dictionary called <code> frequency_matrix</code> is created to store the frequency of the words
- <strong> Stopwords & PorterStemmer  </strong> is used to remove the stopwords and stemming the words.
- each sentence is being tokenized into word level and count the tokens and put in the freq_table to store the frequencies of the word.
- Each words are converted into lowercase and then stemmed. stopwords are being ignored in this step. For each word, this block checks if it's already in the freq_table.
- If it is, the frequency count is incremented.
- If not, the word is added to the table with a frequency count of 1.
- The frequency table for each sentence is added to the frequency_matrix with a key. The key is the length of the sentence for now

In [41]:
from nltk.stem import PorterStemmer

def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:len(sent)]] = freq_table

    return frequency_matrix

freq_matrix = _create_frequency_matrix(combined_sentences)
freq_matrix

{'1. A method comprising: using a first reader to take a first reading of an inherent disorder feature of a tag; using at least a second reader to take at least a second reading of the inherent disorder feature of the tag; matching the first reading with at least the second reading; determining one or more acceptance criteria, wherein at least one of the acceptance criteria is based on whether the first reading and the second reading match within a predetermined threshold; accepting the tag if the acceptance criteria are met; and recording a fingerprint for the tag if the tag was accepted.': {'1': 1,
  '.': 2,
  'method': 1,
  'compris': 1,
  ':': 1,
  'use': 2,
  'first': 4,
  'reader': 2,
  'take': 2,
  'read': 6,
  'inher': 2,
  'disord': 2,
  'featur': 2,
  'tag': 5,
  ';': 5,
  'least': 4,
  'second': 4,
  'match': 2,
  'determin': 1,
  'one': 2,
  'accept': 5,
  'criteria': 3,
  ',': 1,
  'wherein': 1,
  'base': 1,
  'whether': 1,
  'within': 1,
  'predetermin': 1,
  'threshold':

<strong> TF Matrix Calcuation </strong>
- <code> tf_matrix </code> dictionary is created to store the term frequency for each word in each sentence.
- loop over the freq_matrix. sent will be the sentence, and f_table is the corresponding frequency table for that sentence.
- For each sentence, a new empty dictionary <code> tf_table </code> is created. This will store the term frequencies for words in this particular sentence.
- The function then iterates over each word and its count in the frequency table. It calculates the term frequency by dividing the count of each word by the total word count in the sentence, and stores this value in <code> tf_table. </code>
- The term frequency table <code> tf_table </code> for each sentence is then stored in the  <code> tf_matrix </code> with the sentence as the key.

In [42]:
def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

In [43]:
tf_matrix = _create_tf_matrix(freq_matrix)

---

### IDF Calculation

we calculate, “how many sentences contain a word”, Let’s call it Documents per words matrix.

In [44]:
def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

count_doc_per_words = _create_documents_per_words(freq_matrix)

In [45]:
count_doc_per_words

{'1': 7,
 '.': 22,
 'method': 20,
 'compris': 19,
 ':': 7,
 'use': 7,
 'first': 10,
 'reader': 11,
 'take': 3,
 'read': 9,
 'inher': 4,
 'disord': 4,
 'featur': 3,
 'tag': 8,
 ';': 3,
 'least': 6,
 'second': 10,
 'match': 3,
 'determin': 9,
 'one': 8,
 'accept': 8,
 'criteria': 3,
 ',': 20,
 'wherein': 14,
 'base': 7,
 'whether': 3,
 'within': 4,
 'predetermin': 4,
 'threshold': 4,
 'met': 2,
 'record': 2,
 'fingerprint': 2,
 'wa': 1,
 '2': 3,
 'claim': 19,
 'criterion': 4,
 'individu': 3,
 '3': 1,
 'strength': 1,
 'signal': 2,
 '4': 1,
 'complex': 1,
 '5': 3,
 'reject': 5,
 '6': 2,
 'remov': 2,
 'without': 1,
 'stop': 1,
 'flow': 1,
 'product': 1,
 '7': 1,
 'mark': 1,
 'cut': 1,
 'punch': 1,
 'suction': 1,
 '8': 1,
 'note': 1,
 'databas': 1,
 '9': 1,
 'third': 7,
 '10': 2,
 '11': 2,
 'check': 2,
 'perform': 2,
 '12': 1,
 'provid': 1,
 'differ': 1,
 'two': 1,
 '13': 4,
 'vari': 4,
 'condit': 4,
 'cover': 2,
 'rang': 2,
 '14': 1,
 'age': 1,
 'temperatur': 1,
 'construct': 1,
 'compon': 

The result means: 1 appear in 9 sentence and antioxid appears in 12 sentences etc etc...

after that, calcuate idf according to the formula <code> IDF(t) = log_e(Total number of documents / Number of documents with term t in it) </code> which is the number from the above result.

In [46]:
def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix
idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words,total_docs)

In [47]:
idf_matrix

{'1. A method comprising: using a first reader to take a first reading of an inherent disorder feature of a tag; using at least a second reader to take at least a second reading of the inherent disorder feature of the tag; matching the first reading with at least the second reading; determining one or more acceptance criteria, wherein at least one of the acceptance criteria is based on whether the first reading and the second reading match within a predetermined threshold; accepting the tag if the acceptance criteria are met; and recording a fingerprint for the tag if the tag was accepted.': {'1': 0.7676858167054786,
  '.': 0.2703611758975292,
  'method': 0.31175386105575426,
  'compris': 0.33403025576690654,
  ':': 0.7676858167054786,
  'use': 0.7676858167054786,
  'first': 0.6127838567197355,
  'reader': 0.5713911715615104,
  'take': 1.135662602000073,
  'read': 0.6585413472804106,
  'inher': 1.0107238653917732,
  'disord': 1.0107238653917732,
  'featur': 1.135662602000073,
  'tag': 

### Calcualte TFIDF

In [48]:
def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)

In [49]:
tf_idf_matrix

{'1. A method comprising: using a first reader to take a first reading of an inherent disorder feature of a tag; using at least a second reader to take at least a second reading of the inherent disorder feature of the tag; matching the first reading with at least the second reading; determining one or more acceptance criteria, wherein at least one of the acceptance criteria is based on whether the first reading and the second reading match within a predetermined threshold; accepting the tag if the acceptance criteria are met; and recording a fingerprint for the tag if the tag was accepted.': {'1': 0.023263206566832687,
  '.': 0.016385525811971467,
  'method': 0.00944708669865922,
  'compris': 0.010122128962633531,
  ':': 0.023263206566832687,
  'use': 0.04652641313366537,
  'first': 0.0742768311175437,
  'reader': 0.034629767973424874,
  'take': 0.06882803648485292,
  'read': 0.11973479041462011,
  'inher': 0.06125599184192565,
  'disord': 0.06125599184192565,
  'featur': 0.06882803648

### Score the sentence

In [50]:
def _score_sentences(tf_idf_matrix) -> dict:
    """
    score a sentence by its word's TF
    Basic algorithm: adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence

    return sentenceValue

sentenceValue= _score_sentences(tf_idf_matrix)

In [51]:
sentenceValue

{'1. A method comprising: using a first reader to take a first reading of an inherent disorder feature of a tag; using at least a second reader to take at least a second reading of the inherent disorder feature of the tag; matching the first reading with at least the second reading; determining one or more acceptance criteria, wherein at least one of the acceptance criteria is based on whether the first reading and the second reading match within a predetermined threshold; accepting the tag if the acceptance criteria are met; and recording a fingerprint for the tag if the tag was accepted.': 0.05323332989145503,
 '2. The method of claim 1, wherein determining one or more acceptance criteria further comprises: determining an acceptance criterion based on an individual reading.': 0.04541322207271596,
 '3. The method of claim 2, wherein determining an acceptance criterion based on an individual reading comprises determining an acceptance criterion based on a strength of a signal in at lea

In [52]:
def _find_average_score(sentenceValue) -> int:
    """
    Find the average score from the sentence value dictionary
    :rtype: int
    """
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentenceValue))

    return average
threshold = _find_average_score(sentenceValue)

In [53]:
threshold

0.09004998881243623

In [54]:
def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:len(sentence)] in sentenceValue and sentenceValue[sentence[:len(sentence)]] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary

### Generate the summary with tfidf

In [55]:
_generate_summary(combined_sentences, sentenceValue, threshold)

' 14. The method of claim 13, wherein varying the conditions comprises varying at least one of the age of at least one of the readers, the temperature conditions for at least one of the readers, the construction of at least one of the readers, and the components of at least one of the readers. 20-25. (canceled) 26.'

---

## Producing Dependency Graph

---

## Evaluation